In [42]:
#import packages
import numpy as np
import pandas as pd

In [43]:
#read in gz file
df = pd.read_csv('reviews_01.tsv.gz', compression='gzip', header=0, sep='\t',  error_bad_lines=False)

b'Skipping line 5782: expected 15 fields, saw 22\nSkipping line 31979: expected 15 fields, saw 22\nSkipping line 32408: expected 15 fields, saw 22\nSkipping line 45709: expected 15 fields, saw 22\nSkipping line 64585: expected 15 fields, saw 22\n'
b'Skipping line 115742: expected 15 fields, saw 22\nSkipping line 125537: expected 15 fields, saw 22\nSkipping line 127580: expected 15 fields, saw 22\nSkipping line 128696: expected 15 fields, saw 22\n'
b'Skipping line 143851: expected 15 fields, saw 22\nSkipping line 147069: expected 15 fields, saw 22\nSkipping line 149233: expected 15 fields, saw 22\nSkipping line 167536: expected 15 fields, saw 22\nSkipping line 168535: expected 15 fields, saw 22\nSkipping line 171659: expected 15 fields, saw 22\nSkipping line 184533: expected 15 fields, saw 22\nSkipping line 184624: expected 15 fields, saw 22\nSkipping line 184908: expected 15 fields, saw 22\n'
b'Skipping line 219150: expected 15 fields, saw 22\nSkipping line 223178: expected 15 fields, 

In [44]:
#preview dataframe
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,45610553,RMDCHWD0Y5OZ9,B00HH62VB6,618218723,AGPtek® 10 Isolated Output 9V 12V 18V Guitar P...,Musical Instruments,3,0,1,N,N,Three Stars,"Works very good, but induces ALOT of noise.",2015-08-31
1,US,14640079,RZSL0BALIYUNU,B003LRN53I,986692292,Sennheiser HD203 Closed-Back DJ Headphones,Musical Instruments,5,0,0,N,Y,Five Stars,Nice headphones at a reasonable price.,2015-08-31
2,US,6111003,RIZR67JKUDBI0,B0006VMBHI,603261968,AudioQuest LP record clean brush,Musical Instruments,3,0,1,N,Y,Three Stars,removes dust. does not clean,2015-08-31
3,US,1546619,R27HL570VNL85F,B002B55TRG,575084461,Hohner Inc. 560BX-BF Special Twenty Harmonica,Musical Instruments,5,0,0,N,Y,I purchase these for a friend in return for pl...,I purchase these for a friend in return for pl...,2015-08-31
4,US,12222213,R34EBU9QDWJ1GD,B00N1YPXW2,165236328,Blue Yeti USB Microphone - Blackout Edition,Musical Instruments,5,0,0,N,Y,Five Stars,This is an awesome mic!,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903999,US,51647170,R6MDTUK723YPY,B00002DDSF,530155260,Yamaha PSR240 61-Note Touch-Sensitive Portable...,Musical Instruments,4,143,148,N,N,Great keyboard for children,"This keyboard is great, and my children have r...",2000-01-11
904000,US,52108449,R10B4417NCK98O,B00002JV63,70258872,Yamaha PSR340 61-Note Touch-Sensitive Portable...,Musical Instruments,5,24,24,N,N,Delighted,I purchased the PSR340 as a Christmas gift for...,1999-12-29
904001,US,50610066,R3V2A0GUZ3JAQ4,B00002JV63,70258872,Yamaha PSR340 61-Note Touch-Sensitive Portable...,Musical Instruments,4,27,29,N,N,no power supply,"This is a fine keyboard. Unfortunately, Yamah...",1999-12-22
904002,US,50522024,R1VYXC7FFG4LGH,B00002F2IZ,615125621,Yamaha DD9M Touch-Sensitive Digital Drums,Musical Instruments,4,46,49,N,N,I've got the beat!,My son who is a drummer found this very easy t...,1999-12-17


In [45]:
#get number of entries
len(df)

904004

In [46]:
#check for duplicates
df.drop_duplicates(inplace=True)

In [47]:
#no duplicate items
len(df)

904004

In [48]:
#display information about the first item
df.iloc[0]

marketplace                                                         US
customer_id                                                   45610553
review_id                                                RMDCHWD0Y5OZ9
product_id                                                  B00HH62VB6
product_parent                                               618218723
product_title        AGPtek® 10 Isolated Output 9V 12V 18V Guitar P...
product_category                                   Musical Instruments
star_rating                                                          3
helpful_votes                                                        0
total_votes                                                          1
vine                                                                 N
verified_purchase                                                    N
review_headline                                            Three Stars
review_body                Works very good, but induces ALOT of noise.
review

In [49]:
df.dtypes

marketplace          object
customer_id           int64
review_id            object
product_id           object
product_parent        int64
product_title        object
product_category     object
star_rating           int64
helpful_votes         int64
total_votes           int64
vine                 object
verified_purchase    object
review_headline      object
review_body          object
review_date          object
dtype: object

In [65]:
df['product_id'] = df['product_id'].astype('string')

In [50]:
#subset data to help iterrows
rdf = df[['customer_id', 'product_id']]

In [51]:
#create dictionary data structures
from collections import defaultdict

itemsReviewedByEachUser = defaultdict(list)
usersWhoReviewedEachItem = defaultdict(list)

for index, row in rdf.iterrows():
  itemsReviewedByEachUser[row['customer_id']].append(row['product_id'])
  usersWhoReviewedEachItem[row['product_id']].append(row['customer_id'])

In [52]:
#def jaccard
def jaccard(a, b):
  intersect = list(set(a).intersection(b))
  intersectionLen = len(intersect)
  union = list(set(a).union(b))
  unionLen = len(union)
  return (intersectionLen / unionLen)

In [53]:
#implement recommender function
from sortedcontainers import SortedDict 

def recommenderFunction(userDict, productDict, product_id, dataframe):
  #users who have already purchased item i
  similarUsers = productDict[product_id]

  #dict to store similarity scores by k/v pair: 'product_id', score
  similarityScores = {}

  #loop through all rows
  for index, row in dataframe.iterrows():
    #only get items other than i
    if(row['product_id'] != product_id):
      #calculate similarity
      similarityScores[row['product_id']] = jaccard(similarUsers, productDict[row['product_id']])

  return similarityScores

In [54]:
#user is looking at product_id = B0006VMBHI
currentProduct = 'B0006VMBHI'

#recommend 10 products from B0006VMBHI
recommendationsDict = recommenderFunction(userDict=itemsReviewedByEachUser, productDict=usersWhoReviewedEachItem, product_id=currentProduct, dataframe=rdf)

#sort recommendations dict to show highest jaccard
sortedRecommendationList = sorted(recommendationsDict.items(), key=lambda x: x[1], reverse=True)

#return top 10 product id's
for i in range(10):
  print(sortedRecommendationList[i])

('B00006I5SD', 0.028446389496717725)
('B00006I5SB', 0.01694915254237288)
('B000AJR482', 0.015065913370998116)
('B00E7MVP3S', 0.014204545454545454)
('B001255YL2', 0.008955223880597015)
('B003EIRVO8', 0.008849557522123894)
('B0015VEZ22', 0.008333333333333333)
('B00006I5UH', 0.00821917808219178)
('B00008BWM7', 0.008021390374331552)
('B000H2BC4E', 0.007668711656441718)


In [71]:
#get the product titles for interpretation
titles = {}
for i in range(10):
  prodTuple = sortedRecommendationList[i]
  productId = prodTuple[0]
  productDataFrame = (df[df['product_id'] == productId])
  productDataFrame = productDataFrame.iloc[0]
  titles[i+1] = productDataFrame['product_title']

#print items from 1 to 10 (1 is the highest recommended)
print(titles)

{1: 'Shure SFG-2 Stylus Tracking Force Gauge', 2: 'Shure M97xE High-Performance Magnetic Phono Cartridge', 3: 'ART Pro Audio DJPRE II Phono Turntable Preamplifier', 4: 'Signstek Blue LCD Backlight Digital Long-Playing LP Turntable Stylus Force Scale Gauge Tester', 5: 'Audio Technica AT120E/T Standard Mount Phono Cartridge', 6: 'Technics: 45 Adaptor for Technics 1200 (SFWE010)', 7: 'GruvGlide GRUVGLIDE DJ Package', 8: 'STANTON MAGNETICS Record Cleaner Kit', 9: 'Shure M97xE High-Performance Magnetic Phono Cartridge', 10: 'Behringer PP400 Ultra Compact Phono Preamplifier'}
